In [5]:
!pip install TextBlob

  Obtaining dependency information for TextBlob from https://files.pythonhosted.org/packages/1e/d6/40aa5aead775582ea0cf35870e5a3f16fab4b967f1ad2debe675f673f923/textblob-0.19.0-py3-none-any.whl.metadata
  Obtaining dependency information for nltk>=3.9 from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/ad/e0/8adc550d7169df1d6b9be8ff6019cda5291054a0107760c2f30788b6195f/regex-2025.9.1-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ------------------ ------------------- 20.5/41.5 kB 682.7 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 401.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   --------------- ------------------------ 245.8/624.3 kB 5.1 MB/s eta 0:00:01



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np

# Load IBM dataset CSV
df_ibm = pd.read_csv('data\HRAttrition.csv')

# Assume 'EmployeeNumber' uniquely identifies each employee
employee_ids = df_ibm['EmployeeNumber'].unique()

# Settings for survey simulation
np.random.seed(42)  # reproducibility
num_employees = len(employee_ids)
num_questions = 5  # number of survey questions

# Generate synthetic survey data with scores 1 to 5 per question
survey_data = {
    'EmployeeNumber': employee_ids
}
for q in range(1, num_questions + 1):
    survey_data[f'Survey_Q{q}'] = np.random.randint(1, 6, size=num_employees)

# Create DataFrame of synthetic survey results
df_survey = pd.DataFrame(survey_data)

print(df_survey.head())


   EmployeeNumber  Survey_Q1  Survey_Q2  Survey_Q3  Survey_Q4  Survey_Q5
0               1          4          2          5          5          2
1               2          5          2          1          1          1
2               4          3          3          4          2          5
3               5          5          5          2          4          1
4               7          5          2          2          2          5


In [2]:
# Merge dataframes on 'EmployeeNumber' (common key)
merged_df = df_ibm.merge(df_survey, how='left', on='EmployeeNumber')

# Preview merged dataframe
print(merged_df.head())

# Save merged dataframe to CSV
#merged_df.to_csv('ibm_hr_with_survey.csv', index=False)

#print("Merged dataset saved to 'ibm_hr_with_survey.csv'")


   Age Attrition     BusinessTravel  DailyRate              Department  \
0   41       Yes      Travel_Rarely       1102                   Sales   
1   49        No  Travel_Frequently        279  Research & Development   
2   37       Yes      Travel_Rarely       1373  Research & Development   
3   33        No  Travel_Frequently       1392  Research & Development   
4   27        No      Travel_Rarely        591  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
0                 1          2  Life Sciences              1               1   
1                 8          1  Life Sciences              1               2   
2                 2          2          Other              1               4   
3                 3          4  Life Sciences              1               5   
4                 2          1        Medical              1               7   

   ...  WorkLifeBalance YearsAtCompany  YearsInCurrentRole  \
0  ...      

Add free text engagemnt survey comments, do a sentiment analysis and score the engagement 

In [7]:

import pandas as pd
import numpy as np
import random
from textblob import TextBlob  # for simple sentiment analysis

# Load IBM dataset CSV
#df_ibm = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')
#employee_ids = df_ibm['EmployeeNumber'].unique()

# Sample phrases on different topics
phrases = [
    # Work-life balance
    "I balance my work and personal life well.",
    "Long hours make it hard to spend time with family.",
    "Flexible working hours help me stay motivated.",
    # Career plans
    "I have clear goals for my career advancement.",
    "Uncertain about my future path here.",
    "The company supports my growth plans.",
    # Learning
    "I frequently attend learning sessions.",
    "Training opportunities are limited.",
    "I enjoy the learning culture here.",
    # Career path
    "My career path aligns with my interests.",
    "I see limited progression options.",
    "I'm encouraged to explore new roles."
]

# Assign random phrase to each employee
np.random.seed(42)
assigned_phrases = np.random.choice(phrases, size=len(employee_ids))

# Create DataFrame with EmployeeNumber and assigned phrase
df_text = pd.DataFrame({
    'EmployeeNumber': employee_ids,
    'Survey_Text_Response': assigned_phrases
})

# Function to map polarity (-1 to +1) to score 1-5
def polarity_to_score(polarity):
    # Normalize polarity from [-1,1] to [1,5]
    return int(round((polarity + 1) * 2 + 1))

# Perform sentiment analysis on text and score engagement
df_text['Sentiment_Polarity'] = df_text['Survey_Text_Response'].apply(lambda x: TextBlob(x).sentiment.polarity)
df_text['Engagement_Score'] = df_text['Sentiment_Polarity'].apply(polarity_to_score)

# Show results
print(df_text.head())


   EmployeeNumber                           Survey_Text_Response  \
0               1         I frequently attend learning sessions.   
1               2  I have clear goals for my career advancement.   
2               4             I see limited progression options.   
3               5            Training opportunities are limited.   
4               7           Uncertain about my future path here.   

   Sentiment_Polarity  Engagement_Score  
0            0.100000                 3  
1            0.100000                 3  
2           -0.071429                 3  
3           -0.071429                 3  
4            0.000000                 3  


In [8]:
# Merge dataframes on 'EmployeeNumber' (common key)
merged_df = merged_df.merge(df_text, how='left', on='EmployeeNumber')

In [9]:
print(merged_df.head())

   Age Attrition     BusinessTravel  DailyRate              Department  \
0   41       Yes      Travel_Rarely       1102                   Sales   
1   49        No  Travel_Frequently        279  Research & Development   
2   37       Yes      Travel_Rarely       1373  Research & Development   
3   33        No  Travel_Frequently       1392  Research & Development   
4   27        No      Travel_Rarely        591  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
0                 1          2  Life Sciences              1               1   
1                 8          1  Life Sciences              1               2   
2                 2          2          Other              1               4   
3                 3          4  Life Sciences              1               5   
4                 2          1        Medical              1               7   

   ...  YearsSinceLastPromotion YearsWithCurrManager  Survey_Q1  Survey_Q2

In [10]:
import pandas as pd

df = pd.read_csv('data\HRAttrition.csv')

# Inspect columns to identify or create joining and leaving years
# For example, if 'DateOfJoining' exists, convert to datetime and extract year
if 'DateOfJoining' in df.columns:
    df['JoinYear'] = pd.to_datetime(df['DateOfJoining'], errors='coerce').dt.year
else:
    # If no date, try to infer from 'YearAtCompany' assuming dataset year known
    # For example, assume dataset snapshot year 2020
    snapshot_year = 2020
    df['JoinYear'] = snapshot_year - df['YearsAtCompany']

# For leaving year, assuming if Attrition='Yes' the leaving year is join year + YearsAtCompany
df['LeaveYear'] = df['JoinYear'] + df['YearsAtCompany']

# Get unique sorted years
unique_join_years = sorted(df['JoinYear'].dropna().unique())
unique_leave_years = sorted(df['LeaveYear'].dropna().unique())

print("Unique Join Years:", unique_join_years)
print("Unique Leave Years:", unique_leave_years)

# For API arguments:
from_year = min(unique_join_years)
to_year = max(unique_leave_years)
print(f"API arguments: from_year={from_year}, to_year={to_year}")


Unique Join Years: [np.int64(1980), np.int64(1983), np.int64(1984), np.int64(1986), np.int64(1987), np.int64(1988), np.int64(1989), np.int64(1990), np.int64(1991), np.int64(1993), np.int64(1994), np.int64(1995), np.int64(1996), np.int64(1997), np.int64(1998), np.int64(1999), np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]
Unique Leave Years: [np.int64(2020)]
API arguments: from_year=1980, to_year=2020


In [12]:
import requests
import json

# BLS public API endpoint
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

# API parameters
headers = {'Content-type': 'application/json'}
data = json.dumps({
    "seriesid": ["LNS14000000"],  # Unemployment rate national series
    "startyear": "1980",
    "endyear": "2020",
    "registrationKey": ""  # Optional: Add your API key here if available
})

# Make POST request
response = requests.post(url, data=data, headers=headers)

# Parse JSON response
json_data = response.json()

# Print results summary
if json_data['status'] == 'REQUEST_SUCCEEDED':
    series_data = json_data['Results']['series'][0]['data']
    print(f"Unemployment Rate Data (1980-2020): Total {len(series_data)} points")
    for item in series_data[:30]:  # Show first 5 entries as sample
        print(f"Year {item['year']}-{item['periodName']}: {item['value']}%")
else:
    print("API request failed:", json_data)


Unemployment Rate Data (1980-2020): Total 120 points
Year 1989-December: 5.4%
Year 1989-November: 5.4%
Year 1989-October: 5.3%
Year 1989-September: 5.3%
Year 1989-August: 5.2%
Year 1989-July: 5.2%
Year 1989-June: 5.3%
Year 1989-May: 5.2%
Year 1989-April: 5.2%
Year 1989-March: 5.0%
Year 1989-February: 5.2%
Year 1989-January: 5.4%
Year 1988-December: 5.3%
Year 1988-November: 5.3%
Year 1988-October: 5.4%
Year 1988-September: 5.4%
Year 1988-August: 5.6%
Year 1988-July: 5.4%
Year 1988-June: 5.4%
Year 1988-May: 5.6%
Year 1988-April: 5.4%
Year 1988-March: 5.7%
Year 1988-February: 5.7%
Year 1988-January: 5.7%
Year 1987-December: 5.7%
Year 1987-November: 5.8%
Year 1987-October: 6.0%
Year 1987-September: 5.9%
Year 1987-August: 6.0%
Year 1987-July: 6.1%


In [14]:
import json

with open('bls_unemployment_series_data.json', 'w') as f:
    json.dump(series_data, f, indent=2)

print("Series data saved to 'bls_unemployment_series_data.json'")

Series data saved to 'bls_unemployment_series_data.json'


In [16]:
import json

with open('bls_unemployment_series_data.json', 'r') as f:
    json_data = json.load(f)

print(type(json_data))        # Should print <class 'list'> or <class 'dict'>
print(json_data[:2])          # Print first 2 elements if it is a list


<class 'list'>
[{'year': '1989', 'period': 'M12', 'periodName': 'December', 'value': '5.4', 'footnotes': [{}]}, {'year': '1989', 'period': 'M11', 'periodName': 'November', 'value': '5.4', 'footnotes': [{}]}]


In [19]:
import json
import pandas as pd

# Replace 'data.json' with your JSON filename
with open('bls_unemployment_series_data.json', 'r') as f:
    json_data = json.load(f)

# Inspect top-level keys (headers)
print(f"JSON root type: {type(json_data)}") 



# Convert to DataFrame for easier viewing
bls_data = pd.DataFrame(json_data)

# Display top 20 rows
print(bls_data.head(20))


JSON root type: <class 'list'>
    year period periodName value footnotes
0   1989    M12   December   5.4      [{}]
1   1989    M11   November   5.4      [{}]
2   1989    M10    October   5.3      [{}]
3   1989    M09  September   5.3      [{}]
4   1989    M08     August   5.2      [{}]
5   1989    M07       July   5.2      [{}]
6   1989    M06       June   5.3      [{}]
7   1989    M05        May   5.2      [{}]
8   1989    M04      April   5.2      [{}]
9   1989    M03      March   5.0      [{}]
10  1989    M02   February   5.2      [{}]
11  1989    M01    January   5.4      [{}]
12  1988    M12   December   5.3      [{}]
13  1988    M11   November   5.3      [{}]
14  1988    M10    October   5.4      [{}]
15  1988    M09  September   5.4      [{}]
16  1988    M08     August   5.6      [{}]
17  1988    M07       July   5.4      [{}]
18  1988    M06       June   5.4      [{}]
19  1988    M05        May   5.6      [{}]


In [20]:
import json
import pandas as pd

# Load BLS JSON data from file
with open('bls_unemployment_series_data.json', 'r') as f:
    json_data = json.load(f)

# The root is a list of monthly records with keys: 'year', 'period', 'periodName', 'value', 'footnotes'
# Convert to DataFrame
df_bls = pd.DataFrame(json_data)

# Convert 'value' column to float (unemployment rate)
df_bls['value'] = df_bls['value'].astype(float)

# Aggregate by year to get yearly average unemployment rate
yearly_unemp = df_bls.groupby('year')['value'].mean().reset_index()
yearly_unemp.rename(columns={'year': 'year', 'value': 'UnemploymentRate'}, inplace=True)

# Sort by year for trend calculation
yearly_unemp = yearly_unemp.sort_values('year').reset_index(drop=True)

# Create 'Trend' column - compare unemployment rate against previous year
yearly_unemp['Trend'] = yearly_unemp['UnemploymentRate'].diff().apply(
    lambda x: 'Increasing' if x > 0 else 'Decreasing'
)
# Fill NaN for first year
yearly_unemp['Trend'] = yearly_unemp['Trend'].fillna('No Data')

# Prepare employee DataFrame (example structure)
# Assume df_emp has 'LeaveYear' column (int) representing the year employee left
# For demonstration, creating a sample df_emp
df_emp = pd.DataFrame({
    'EmployeeNumber': [1001, 1002, 1003, 1004],
    'LeaveYear': [1988, 1989, 1987, 1990]
})

# Convert 'LeaveYear' to string to match yearly_unemp 'year' dtype if needed
df_emp['LeaveYear'] = df_emp['LeaveYear'].astype(str)

# Merge employee data with unemployment data on LeaveYear/year
merged_df = pd.merge(
    df_emp,
    yearly_unemp,
    how='left',
    left_on='LeaveYear',
    right_on='year'
)

# Drop redundant 'year' column after merge
merged_df.drop(columns=['year'], inplace=True)

# Show final DataFrame with unemployment rate and trend corresponding to employee leaving year
print(merged_df)


   EmployeeNumber LeaveYear  UnemploymentRate       Trend
0            1001      1988          5.491667  Decreasing
1            1002      1989          5.258333  Decreasing
2            1003      1987          6.175000  Decreasing
3            1004      1990               NaN         NaN
